### **Creación de variables agregadas a partir del dataset del dataset [Cost of Living](https://www.kaggle.com/datasets/mvieira101/global-cost-of-living/code)**

Este notebook implementa la creación de **7 índices calculados** sobre el dataset `cost-of-living-clean.csv`.

La idea general es que variables desagregadas sobre el coste de frutas o verduras por separada aportan poco valor analítico. 

En cambio el cálculo de variables agregadas relacionadas con coste de la cesta de compra o precio de la vivienda sí lo hacen. 

#### **Pasos a seguir y descripción de variables agregadas**

1. **Setup y carga de datos**: Importar librerías y cargar `cost-of-living-clean.csv`
2. **Exploración inicial**: Verificar columnas disponibles y tipos de datos
3. **Variables agregada 1 — `nomad_housing_cost`**: promedio alquiler 1br (centro + afueras) / 2
4. **Variables agregada 2 — `basic_basket_index`**: promedio productos básicos supermercado
5. **Variables agregada 3 — `daily_meal_cost`**: cappuccino + comida restaurante económico
6. **Variables agregada 4 — `monthly_nomad_cost`**: coste mensual total (vivienda + comida + internet + utilities + transporte)
7. **Variables agregada 5 — `local_purchasing_power`**: salario / coste mensual nómada
8. **Variables agregada 6 — `cappuccino_index`**: normalización del precio cappuccino
9. **Variables agregada 7 — `housing_salary_ratio`**: (alquiler / salario) × 100
10. **Creación del nuevo CSV con la variables agregadas**: creación de nuevas columnas y exportar CSV actualizado

#### **Dependencias entre variables agregadas**

- **Independientes**: las variables agregadas 1, 2, 3 y 6 se pueden crear en paralelo.
- **Variable agregada 4**: depende de las variables 1, 2 y 3.
- **Variable agregada 5**: depende de la variable 4
- **Variable agregada 7**: depende de la variable 1

#### **Hipótesis que validan**

1. La variable `nomad_housing_cost` está relacionada con la verificación de las hipótesis 1 y 4 del planteamiento del EDA. 
2. La variable `basic_basket_index` con las hipótesis 1 y 2. 
3. La variable `daily_meal_cost` con la hipótesis 1. 
4. La variable `monthly_nomad_cost` con las hipótesis 1 y 5. 
5. La variable `local_purchasing_power` con la hipótesis 2. 
6. La variable `cappuccino_index` con la hipótesis 1. 
7. La variable `housing_salary_ratio` con las hipótesis 2 y 4. 

### 1. **Importar librerías y cargar el dataset de Cost of Living**

In [2]:
import numpy as np
import pandas as pd

pd.options.mode.copy_on_write = True # CoW por defecto a partir de pandas 3.0.0 

In [3]:
df_cost = pd.read_csv("./data/cost-of-living-clean.csv")
df_cost.head(20) # Cargamos el CSV limpios de Cost of Living y visualizamos 20 filas y tenerlo cargado

,city_name,country_name,meal_inexpensive_restaurant,meal_midrange_restaurant_2p,mcmeal_fastfood,beer_domestic_restaurant_0_5l,beer_imported_restaurant_0_33l,cappuccino_restaurant,soda_restaurant_0_33l,water_restaurant_0_33l,...,rent_1br_city_center,rent_1br_outside_center,rent_3br_city_center,rent_3br_outside_center,price_sqm_city_center,price_sqm_outside_center,avg_net_salary,mortgage_interest_rate_20y,data_quality_flag,continent
0,Seoul,South Korea,7.68,53.78,6.15,3.07,4.99,3.93,1.48,0.79,...,742.54,557.52,2669.12,1731.08,22067.70,10971.90,2689.62,3.47,1,Asia
1,Shanghai,China,5.69,39.86,5.69,1.14,4.27,3.98,0.53,0.33,...,1091.93,569.88,2952.70,1561.59,17746.11,9416.35,1419.87,5.03,1,Asia
2,Guangzhou,China,4.13,28.47,4.98,0.85,1.71,3.54,0.44,0.33,...,533.28,317.45,1242.24,688.05,12892.82,5427.45,1211.68,5.19,1,Asia
3,Mumbai,India,3.68,18.42,3.68,2.46,4.30,2.48,0.48,0.19,...,522.40,294.05,1411.12,699.80,6092.45,2777.51,640.81,7.96,1,Asia
4,Delhi,India,4.91,22.11,4.30,1.84,3.68,1.77,0.49,0.19,...,229.84,135.31,601.02,329.15,2506.73,1036.74,586.46,8.06,1,Asia
5,Dhaka,Bangladesh,1.95,11.71,4.88,5.85,5.12,1.95,0.29,0.16,...,142.09,87.79,347.57,208.50,1119.98,571.72,280.73,9.26,1,Asia
6,Osaka,Japan,7.45,48.39,5.36,3.35,3.72,3.28,1.09,0.81,...,674.96,376.14,1737.21,993.17,8043.38,4825.58,2322.46,1.49,1,Asia
7,Jakarta,Indonesia,2.59,22.69,3.57,2.06,3.24,2.23,0.61,0.27,...,505.59,277.43,1172.14,615.04,2632.80,1241.09,509.12,9.05,1,Asia
8,Shenzhen,China,4.27,28.47,4.98,1.14,3.99,4.20,0.47,0.34,...,738.75,435.07,1682.30,886.16,17898.73,8091.57,1572.22,4.99,1,Asia
9,Kinshasa,Congo,15.11,42.63,10.08,1.74,2.50,4.35,2.78,0.84,...,2000.00,725.00,4500.00,1160.00,6170.63,933.33,400.00,19.33,0,Africa


#### **2. Calculamos la variable agregada 1: `nomad_housing_cost`**

Coste medio de alquiler para un nómada digital (1 dormitorio).

¿Por qué 1 dormitorio y precios relacionados con el alquiler? 

En general los nómadas digital prefieren alquilar y no comprar por movilidad constante y suelen hacerlo en etapas de su vida sin familia.

En este sentido, no usamos las variables de alquiler de pisos de 3 dormitorios y el coste de comprar una vivienda. 

**La variable agregada sería sima de los costes de alquiler de vivienda de 1 dormitorio en el centro de la ciudad y a la afueras**: 

**Cálculo variable agregada**: `(rent_1br_city_center + rent_1br_outside_center) / 2`

In [4]:
# Promedio entre alquiler en centro y afueras (1 dormitorio)
# Representa el coste típico de vivienda para un nómada digital

df_cost['nomad_housing_cost'] = (
    df_cost['rent_1br_city_center'] + 
    df_cost['rent_1br_outside_center']
) / 2

# Valores estadísticos estándar de la nueva variable que usaremos en el EDA 

print(f"Valores estadísticos de la variable agregada relacionada con alquiler de vivienda de 1 dormitorio")
print(f"-------------------------------------------------------------------------------------------------")
print(df_cost['nomad_housing_cost'].describe())

Valores estadísticos de la variable agregada relacionada con alquiler de vivienda de 1 dormitorio
-------------------------------------------------------------------------------------------------
count     4742.000000
mean       636.678457
std        553.182086
min         18.985000
25%        219.809375
50%        471.722500
75%        945.632500
max      10799.100000
Name: nomad_housing_cost, dtype: float64


#### **3. Calculamos la variable agregada 2: `basic_basket_index`**

Coste promedio productos básicos supermercado;

¿Por qué es importante tener en cuenta el precio de una canasta básica?

El costo de los productos básicos es uno de los gastos principales para las personas nómadas digitales. Analizar esta variable permite comparar ciudades y estimar dónde es más accesible cubrir necesidades esenciales de alimentación, identificando lugares con canastas básicas más completas y a precios más asequibles.


In [ ]:
# Identificamos que columnas "parecen" de supermercado 

num_cols = df_cost.select_dtypes(include="number").columns
list(num_cols)

['meal_inexpensive_restaurant',
 'meal_midrange_restaurant_2p',
 'mcmeal_fastfood',
 'beer_domestic_restaurant_0_5l',
 'beer_imported_restaurant_0_33l',
 'cappuccino_restaurant',
 'soda_restaurant_0_33l',
 'water_restaurant_0_33l',
 'milk_1l',
 'bread_white_500g',
 'rice_white_1kg',
 'eggs_12',
 'cheese_local_1kg',
 'chicken_fillet_1kg',
 'beef_1kg',
 'apples_1kg',
 'bananas_1kg',
 'oranges_1kg',
 'tomatoes_1kg',
 'potatoes_1kg',
 'onions_1kg',
 'lettuce_1unit',
 'water_1_5l_supermarket',
 'wine_midrange_supermarket',
 'beer_domestic_supermarket_0_5l',
 'beer_imported_supermarket_0_33l',
 'cigarettes_pack_marlboro',
 'public_transport_ticket_one_way',
 'public_transport_monthly_pass',
 'taxi_start_fare',
 'taxi_per_km',
 'taxi_waiting_1h',
 'gasoline_1l',
 'car_vw_golf_new',
 'car_toyota_corolla_new',
 'utilities_85sqm',
 'mobile_prepaid_1min',
 'internet_60mbps_unlimited',
 'gym_monthly_membership',
 'cinema_ticket',
 'private_preschool_monthly',
 'international_primary_school_yearly'

In [ ]:
# Visualizamos 20 filas para tenerlo cargado con los alimentos
 
grocery_cols = [
    "milk_1l","bread_white_500g","rice_white_1kg","eggs_12","cheese_local_1kg",
    "chicken_fillet_1kg","beef_1kg","apples_1kg","bananas_1kg","oranges_1kg",
    "tomatoes_1kg","potatoes_1kg","onions_1kg","water_1_5l_supermarket",
    "beer_domestic_supermarket_0_5l"
]

df_cost[["city_name", "country_name"] + grocery_cols].head(20)

,city_name,country_name,milk_1l,bread_white_500g,rice_white_1kg,eggs_12,cheese_local_1kg,chicken_fillet_1kg,beef_1kg,apples_1kg,bananas_1kg,oranges_1kg,tomatoes_1kg,potatoes_1kg,onions_1kg,water_1_5l_supermarket,beer_domestic_supermarket_0_5l
0,Seoul,South Korea,2.20,2.85,3.53,4.04,11.54,10.58,41.61,6.77,3.71,6.50,6.19,3.84,2.92,1.05,2.12
1,Shanghai,China,2.74,2.61,1.22,2.22,18.35,4.86,13.12,2.26,1.60,2.19,1.53,0.84,1.04,0.64,0.94
2,Guangzhou,China,1.91,1.63,1.03,1.71,9.00,3.77,11.75,2.02,1.44,1.82,1.31,0.74,1.00,0.51,0.95
3,Mumbai,India,0.75,0.50,0.83,0.95,5.88,3.69,5.95,2.09,0.67,1.34,0.59,0.44,0.44,0.35,2.27
4,Delhi,India,0.73,0.50,0.85,1.02,4.36,3.81,5.71,1.79,0.75,1.03,0.61,0.37,0.41,0.36,1.54
5,Dhaka,Bangladesh,0.83,0.67,0.69,1.32,7.21,3.07,7.19,2.38,1.04,2.18,0.96,0.28,0.49,0.27,2.40
6,Osaka,Japan,1.41,1.47,4.92,1.90,11.09,6.93,22.46,4.09,1.85,3.87,6.34,3.31,2.01,0.98,2.54
7,Jakarta,Indonesia,1.30,1.20,0.83,1.71,6.90,3.52,8.49,2.99,1.60,2.13,1.30,1.42,2.05,0.44,2.17
8,Shenzhen,China,2.23,2.40,1.00,2.13,13.67,4.37,15.56,1.97,1.47,1.81,1.45,1.06,1.16,0.55,0.88
9,Kinshasa,Congo,2.00,1.33,5.17,4.15,9.50,5.00,20.00,10.00,3.25,5.25,6.33,3.60,2.83,2.00,1.32


In [ ]:
# Ajustamos cantidades aproximadas de un consumo pequeño mensual ya que estan mezcladas unidades con litros, gramos y kg 
# Consideramos que es una canasta muy basica y entendiendo que faltan productos, como cafe, azucar, papa, legumbres, verduras, etc.

weights = {
    "milk_1l": 5,                   #5L/mes
    "bread_white_500g": 20,         #20 panes/mes
    "rice_white_1kg": 4,            #4 kg por mes
    "eggs_12": 3,                   #36 huevos al mes
    "chicken_fillet_1kg": 4,        #4kg por mes
    "beef_1kg": 2,                  #2 kg por mes
    "apples_1kg": 2,                #2 kg por mes
    "bananas_1kg": 2,               #2kg por mes
    "oranges_1kg": 2,               #2kg por mes     
    "tomatoes_1kg": 2,              #2kg por mes
    "potatoes_1kg": 3,              #3kg por mes
    "onions_1kg": 1,                #1kg por mes
    "cheese_local_1kg": 0.5,        #medio kilo/mes
    "water_1_5l_supermarket": 37,   #asumiendo que persona promedio gasta 2l de agua al dia, aqui indica 1.5l
}

df_cost["basic_basket_monthly_est"] = sum(df_cost[col] * w for col, w in weights.items())

In [ ]:
# Comprobamos a "ojo" si tiene coherencia el precio 

df_cost[["city_name", "country_name", "basic_basket_monthly_est"]].head(20)

,city_name,country_name,basic_basket_monthly_est
0,Seoul,South Korea,325.180
1,Shanghai,China,174.695
2,Guangzhou,China,129.750
3,Mumbai,India,73.610
4,Delhi,India,72.150
5,Dhaka,Bangladesh,78.975
6,Osaka,Japan,220.515
7,Jakarta,Indonesia,112.090
8,Shenzhen,China,163.065
9,Kinshasa,Congo,271.770


In [ ]:
# Buscamos ue producto es el que "infla" la canasta basica y vemos que "beef" y "chicken" dominan, asi que comprobamos si es el paìs
# o si es un dato incoherente utilizando el paìs mas caro:

city = "Aarau"  # o Schwyz
cols = ["city_name","country_name"] + list(weights.keys()) + ["basic_basket_monthly_est"]

df_cost[df_cost["city_name"].eq(city)][cols].T

,3128
city_name,Aarau
country_name,Switzerland
milk_1l,1.89
bread_white_500g,3.05
rice_white_1kg,2.19
eggs_12,5.64
chicken_fillet_1kg,33.12
beef_1kg,96.16
apples_1kg,4.06
bananas_1kg,2.56


In [ ]:
# Y el País más barato:

city = "Chiniot"  # o Pakistan
cols = ["city_name","country_name"] + list(weights.keys()) + ["basic_basket_monthly_est"]

df_cost[df_cost["city_name"].eq(city)][cols].T

,914
city_name,Chiniot
country_name,Pakistan
milk_1l,0.38
bread_white_500g,0.24
rice_white_1kg,0.51
eggs_12,0.91
chicken_fillet_1kg,1.7
beef_1kg,2.68
apples_1kg,0.49
bananas_1kg,0.31


In [83]:
# Buscamos ranking de canasta basica mas baratas
df_cost.sort_values("basic_basket_monthly_est").head(20)[
    ["city_name","country_name","basic_basket_monthly_est"]
]

,city_name,country_name,basic_basket_monthly_est
914,Chiniot,Pakistan,37.450
4198,Baglung,Nepal,40.480
1244,Mardan,Pakistan,41.650
1006,Jhang City,Pakistan,42.400
3261,Krishnapur,India,42.480
2666,El Kef,Tunisia,42.840
648,Bahawalpur,Pakistan,43.560
2508,Tataouine,Tunisia,44.065
2925,Matale,Sri Lanka,44.235
718,Sargodha,Pakistan,44.390


In [ ]:
# Buscamos ranking a canasta mensual mas cara

df_cost.sort_values("basic_basket_monthly_est", ascending=False).head(20)[
    ["city_name","country_name","basic_basket_monthly_est"]
]

,city_name,country_name,basic_basket_monthly_est
529,Hamilton,Bermuda,579.220
4355,Gallup,United States,537.685
3128,Aarau,Switzerland,520.715
4725,Wrightsville,United States,514.955
2743,East Orange,United States,513.875
3215,Schwyz,Switzerland,511.770
2991,South Miami Heights,United States,507.090
3998,North Brunswick,United States,503.145
4652,Birch Bay,United States,498.445
3657,Saratoga Springs,United States,480.970


In [ ]:
# Aqui le pedi a chat me filtrara por rango intermedio 40, 60, ¿ Y por que ese rango?

# Porque es una forma práctica de quedarse con el “centro” de la distribución y evitar tanto los valores muy bajos 
# como los muy altos.

# Percentil 40 = el valor por debajo del cual está el 40% de las ciudades (más baratas).

# Percentil 60 = el valor por debajo del cual está el 60% de las ciudades.

# Entonces, entre 40 y 60 te quedas con el 20% más “intermedio” (las ciudades alrededor de la mediana),
#  que suele representar “ni caro ni barato”.

low_cost_montly = df_cost["basic_basket_monthly_est"].quantile(0.40)
high_cost_monthly = df_cost["basic_basket_monthly_est"].quantile(0.60)

df_cost.loc[
    df_cost["basic_basket_monthly_est"].between(low_cost_montly, high_cost_monthly),
    ["city_name","country_name","basic_basket_monthly_est"]
].head(20)

,city_name,country_name,basic_basket_monthly_est
1,Shanghai,China,174.695
8,Shenzhen,China,163.065
10,Bangkok,Thailand,142.095
13,Sao Paulo,Brazil,131.005
14,Mexico City,Mexico,156.880
15,Lagos,Nigeria,149.250
17,Beijing,China,154.270
18,Moscow,Russia,137.940
27,Chengdu,China,146.940
29,Wuhan,China,141.305


In [ ]:
#Buscamos la mediana para que los datos suenen mas coherentes

median = df_cost["basic_basket_monthly_est"].median()

mid2 = df_cost.assign(
    dist=(df_cost["basic_basket_monthly_est"] - median).abs()
).sort_values("dist")[["city_name","country_name","basic_basket_monthly_est"]]

mid2.head(20)

,city_name,country_name,basic_basket_monthly_est
2801,Bac Giang,Vietnam,152.575
2210,Bonao,Dominican Republic,152.490
3313,Ta' Xbiex,Malta,152.485
2238,Ciudad Hidalgo,Mexico,152.630
3091,Ennis,Ireland,152.435
2889,Figueras,Spain,152.650
3391,Imdina,Malta,152.415
4388,Varadero,Cuba,152.685
3944,Etten-Leur,Netherlands,152.695
410,San Pedro Sula,Honduras,152.700


In [92]:
# Valores estadísticos 

print(f"Valores estadísticos de la variable agregada relacionada con el costo mensual de canasta basica.")
print(df_cost["basic_basket_monthly_est"].describe())

Valores estadísticos de la variable agregada relacionada con el costo mensual de canasta basica.
count    4742.000000
mean      173.880255
std        85.660518
min        37.450000
25%       107.925000
50%       152.532500
75%       234.680000
max       579.220000
Name: basic_basket_monthly_est, dtype: float64


 Aquí podriamos empezar a hacer comparativas de que paises son mas costosos o asequibles pero todo depende del sueldo de un nomada, o la duracion de su estancia, ya que la pregunta es:

**¿Que sugiririamos a un Nómada segun a datos de los paises mas asequibles o no y su relacion a una calidad de vida balanceada?**

#### **4. Calculamos la variable agregada 3 : `daily_meal_cost`**

Cappuccino + comida restaurante económico

Costo estimado de una comida intermitente fuera de casa, calculado como la suma del precio de una comida en restaurante económico y un cappuccino. Esta variable aproxima un gasto típico en días en los que una persona nómada digital no puede cocinar (por falta de tiempo, alojamiento sin cocina, trabajo en coworking o necesidad de comer fuera mientras se mueve) y permite comparar ciudades en términos de accesibilidad para cubrir una comida básica fuera de casa.

**Cálculo variable agregada**: `meal_inexpensive_restaurant + cappuccino_restaurant`

In [86]:
df_cost['daily_meal_cost'] = (
    df_cost['meal_inexpensive_restaurant'] + 
    df_cost['cappuccino_restaurant']
)

df_cost["daily_meal_cost"].describe()

count    4742.000000
mean       13.135526
std         7.959227
min         1.100000
25%         6.372500
50%        12.255000
75%        18.920000
max        64.280000
Name: daily_meal_cost, dtype: float64

In [87]:
#Para tener mas claro añado un top 10 de ciudades mas caras y considerando que es un costo por día

df_cost[["city_name","country_name","daily_meal_cost"]].sort_values("daily_meal_cost", ascending=False).head(10)

,city_name,country_name,daily_meal_cost
1019,Turkmenabat,Turkmenistan,64.28
1525,Dasoguz,Turkmenistan,62.85
1785,Lorain,United States,58.25
3771,Lake Havasu City,United States,56.50
2991,South Miami Heights,United States,56.00
3266,Summit,United States,55.50
4328,Cranford,United States,54.50
3807,Wayne,United States,54.00
4658,Mechanicsburg,United States,53.83
4695,Lihue,United States,50.00


In [93]:
# Y un top 10 de los mas asequibles

df_cost[["city_name","country_name","daily_meal_cost"]].sort_values("daily_meal_cost", ascending=True).head(10)

,city_name,country_name,daily_meal_cost
918,Shekhupura,Pakistan,1.10
718,Sargodha,Pakistan,1.12
1244,Mardan,Pakistan,1.12
621,Akure,Nigeria,1.13
1381,Djelfa,Algeria,1.20
1635,Mandi Burewala,Pakistan,1.20
2335,Jendouba,Tunisia,1.29
2129,Laghouat,Algeria,1.30
2368,Gafsa,Tunisia,1.30
2261,Ghardaia,Algeria,1.33


In [94]:
# Buscamos un balance 
low = df_cost["daily_meal_cost"].quantile(0.40)
high = df_cost["daily_meal_cost"].quantile(0.60)

df_cost.loc[
    df_cost["daily_meal_cost"].between(low, high),
    ["city_name","country_name","daily_meal_cost"]
].head(20)

,city_name,country_name,daily_meal_cost
0,Seoul,South Korea,11.61
1,Shanghai,China,9.67
6,Osaka,Japan,10.73
14,Mexico City,Mexico,10.45
19,Tokyo,Japan,10.87
33,Nagoya,Japan,10.10
43,Luanda,Angola,11.77
62,Hong Kong,Hong Kong,12.79
66,Santiago,Chile,11.67
70,Riyadh,Saudi Arabia,10.49


#### **4. Calculamos la variable agregada 4 : `monthly_nomad_cost`**

Coste mensual total estimado para un nómada digital.

La variable agregada se calcula de una proyección mensual del coste de las siguientes variables:

Vivienda: nomad_housing_cost (la variable agregada 1)

Alimentación: (daily_meal_cost × 8) + (basic_basket_index × 30) (variables agregadas 2 y 3 pero con proyecciones de coste diferentes: 30% días

comer fuera 2 veces por semana y 100% comer en casa)

Internet: internet_60mbps_unlimited

Utilities: utilities_85sqm

Transporte: public_transport_monthly_pass


In [95]:
# Utilizamos el codigo de Juan modificando un poco y después concluimos
# Coste mensual total para un nómada digital, que incluye: 
# 1. Vivienda. 
# 2. Alimentación.
# 3. Internet.
# 4. Suministros tipo gas, luz...
# 5. Transporte.

# Calculamos el coste mensual de alimentación. Aquí concluí que siendo nomada, lo que interesa es gastar lo menos posible, siendo que 
# comer 15 días fuera lo encuentro expensive (se platica en el equipo y modificamos si es necesario) 
# 8 días comiendo fuera (pensando en fines de semana) y 30 días consumiendo la compra mensual en el supermercado)

monthly_food_cost = (
    df_cost['daily_meal_cost'] * 8 +       
    df_cost['basic_basket_monthly_est']     
)

# Calculamos el coste mensual total con todas las variables agregadas y no agregadas. 

df_cost['monthly_nomad_cost'] = (
    df_cost['nomad_housing_cost'] +        
    monthly_food_cost +                      
    df_cost['internet_60mbps_unlimited'] +  
    df_cost['utilities_85sqm'] +            
    df_cost['public_transport_monthly_pass'] 
)

# Datos estadísticos generales de la nueva variable agregada 

print(f"Valores estadísticos de la variable agregada relacionada con el coste mensual total:")
print(f"------------------------------------------------------------------------------------")
print(df_cost['monthly_nomad_cost'].describe())

Valores estadísticos de la variable agregada relacionada con el coste mensual total:
------------------------------------------------------------------------------------
count      4742.000000
mean       1234.800974
std        7095.123143
min         122.325000
25%         511.437500
50%         966.527500
75%        1660.362500
max      487014.575000
Name: monthly_nomad_cost, dtype: float64


In [ ]:
# Los datos estadisticos indican un dato fuera de lo normal, asi que en esta parte buscamos que lo provoca: 

df_cost.nlargest(5, "monthly_nomad_cost")[["city_name","country_name","monthly_nomad_cost",
                                           "nomad_housing_cost","daily_meal_cost",
                                           "internet_60mbps_unlimited","utilities_85sqm",
                                           "public_transport_monthly_pass","basic_basket_monthly_est"]]

,city_name,country_name,monthly_nomad_cost,nomad_housing_cost,daily_meal_cost,internet_60mbps_unlimited,utilities_85sqm,public_transport_monthly_pass,basic_basket_monthly_est
521,Honiara,Solomon Islands,487014.575,627.740,9.52,485991.77,6.07,71.36,241.475
580,Kermanshah,Iran,10986.800,10799.100,6.62,12.28,10.11,9.00,103.350
341,Sharjah,United Arab Emirates,6365.875,5837.510,10.08,91.51,165.33,68.06,122.825
2466,Lae,Papua New Guinea,5324.095,4909.530,10.81,28.45,79.66,38.34,181.635
522,Monaco,Monaco,5321.540,4501.395,30.88,57.60,184.18,37.93,293.395


In [ ]:
# Limpiamos y agregamos a una variable para no tocar el DataFrame original, vemos que habia una sola fila que alteraba los resultados 

df_clean = df_cost[df_cost["internet_60mbps_unlimited"] < 1e5].copy()
df_clean

,city_name,country_name,meal_inexpensive_restaurant,meal_midrange_restaurant_2p,mcmeal_fastfood,beer_domestic_restaurant_0_5l,beer_imported_restaurant_0_33l,cappuccino_restaurant,soda_restaurant_0_33l,water_restaurant_0_33l,...,basic_basket_index_median,daily_meal_cost,monthly_nomad_cost,local_purchasing_power,cappuccino_index,monthly_nomad_cost_fixed,basic_basket_monthly_est,grocery_missing,internet_60mbps_unlimited_fixed,monthly_nomad_cost_est
0,Seoul,South Korea,7.68,53.78,6.15,3.07,4.990,3.93,1.48,0.79,...,4.040,11.61,1314.9500,0.643732,0.379346,4178.1700,325.180,0,22.48,1314.9500
1,Shanghai,China,5.69,39.86,5.69,1.14,4.270,3.98,0.53,0.33,...,2.220,9.67,1194.5000,0.534347,0.384458,2657.2050,174.695,0,17.07,1194.5000
2,Guangzhou,China,4.13,28.47,4.98,0.85,1.710,3.54,0.44,0.33,...,1.710,7.67,721.2550,0.686347,0.339468,1765.4050,129.750,0,16.66,721.2550
3,Mumbai,India,3.68,18.42,3.68,2.46,4.300,2.48,0.48,0.19,...,0.830,6.16,588.9250,0.517235,0.231084,1238.9150,73.610,0,9.33,588.9250
4,Delhi,India,4.91,22.11,4.30,1.84,3.680,1.77,0.49,0.19,...,0.850,6.68,385.8550,0.603413,0.158487,971.9050,72.150,0,7.95,385.8550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4737,Rockhampton,Australia,14.95,64.56,8.15,4.25,3.400,3.40,2.74,2.38,...,2.830,18.35,1273.7850,1.355144,0.325153,2568.7750,197.810,0,42.47,1273.7850
4738,Egilsstadhir,Iceland,17.01,70.87,8.50,4.25,3.540,3.90,1.77,1.42,...,2.550,20.91,1278.5800,0.841491,0.376278,2947.7550,286.825,0,56.70,1278.5800
4739,Ixtapa Zihuatanejo,Mexico,5.16,30.94,12.89,0.98,3.090,1.80,0.62,0.41,...,1.335,6.96,636.5600,0.390469,0.161554,1557.6250,123.085,0,27.46,636.5600
4740,Iqaluit,Canada,29.65,74.27,13.71,6.67,8.890,3.71,3.52,4.08,...,3.860,33.36,2120.0125,0.769762,0.356851,4001.4125,388.100,0,84.00,2120.0125


In [ ]:
# Aplicamos un ranking y buscamos un top 10 de coste mensual mas caro y aqui vemos que hay un dato 'exagerado'

df_clean[["city_name","country_name","monthly_nomad_cost"]].sort_values("monthly_nomad_cost", ascending=False).head(10)

,city_name,country_name,monthly_nomad_cost
580,Kermanshah,Iran,10986.800
341,Sharjah,United Arab Emirates,6365.875
2466,Lae,Papua New Guinea,5324.095
522,Monaco,Monaco,5321.540
2726,Redondo Beach,United States,4188.300
3058,Mount Hagen,Papua New Guinea,4168.200
21,New York,United States,4122.760
529,Hamilton,Bermuda,3988.750
4466,Wailuku,United States,3901.195
3434,Boca Raton,United States,3771.265


In [107]:
# Y un top 10 de los mas asequibles
# Y aqui ignoramos esos valores a 0.0 

df_clean.loc[df_cost["monthly_nomad_cost"] > 0,
            ["city_name","country_name","monthly_nomad_cost"]] \
  .sort_values("monthly_nomad_cost", ascending=True) \
  .head(10)

,city_name,country_name,monthly_nomad_cost
2867,Puttalam,Sri Lanka,122.3250
2456,Dhangadhi,Nepal,134.8450
1614,Moratuwa,Sri Lanka,137.2650
2925,Matale,Sri Lanka,138.6975
4198,Baglung,Nepal,140.4100
3845,Tulsipur,Nepal,140.6150
2591,Kirtipur,Nepal,144.8300
2335,Jendouba,Tunisia,144.9450
3068,Nuwara Eliya,Sri Lanka,148.2225
718,Sargodha,Pakistan,150.2650


In [108]:
#Seguimos la misma logica y hacemos una mediana

# Percentil 40 = el valor por debajo del cual está el 40% de las ciudades (más baratas).

# Percentil 60 = el valor por debajo del cual está el 60% de las ciudades.

# Entonces, entre 40 y 60 te quedas con el 20% más “intermedio” (las ciudades alrededor de la mediana),
#  que suele representar “ni caro ni barato”.

low_price_monthly = df_cost["monthly_nomad_cost"].quantile(0.40)
high_price_monthly = df_cost["monthly_nomad_cost"].quantile(0.60)

df_cost.loc[
    df_cost["monthly_nomad_cost"].between(low_price_monthly, high_price_monthly),
    ["city_name","country_name","monthly_nomad_cost"]
].head(20)

,city_name,country_name,monthly_nomad_cost
1,Shanghai,China,1194.500
6,Osaka,Japan,1081.635
8,Shenzhen,China,939.165
10,Bangkok,Thailand,753.770
13,Sao Paulo,Brazil,796.180
14,Mexico City,Mexico,955.735
66,Santiago,Chile,899.925
69,Nantong,China,912.455
70,Riyadh,Saudi Arabia,1043.690
93,Guadalajara,Mexico,775.850


In [110]:
# Aqui detectamos el por que salieron datos rotos 
df_clean.nlargest(15, "monthly_nomad_cost")[["city_name","country_name","monthly_nomad_cost","data_quality_flag"]]

,city_name,country_name,monthly_nomad_cost,data_quality_flag
580,Kermanshah,Iran,10986.800,0
341,Sharjah,United Arab Emirates,6365.875,1
2466,Lae,Papua New Guinea,5324.095,0
522,Monaco,Monaco,5321.540,0
2726,Redondo Beach,United States,4188.300,0
3058,Mount Hagen,Papua New Guinea,4168.200,0
21,New York,United States,4122.760,1
529,Hamilton,Bermuda,3988.750,1
4466,Wailuku,United States,3901.195,0
3434,Boca Raton,United States,3771.265,0


In [ ]:
#Asi que le pedi a Chat me creara un codigo para no eliminar ninguna fila y solo "arreglar" para que el ranking no se rompa.

s = df_cost["monthly_nomad_cost"]
cap_low, cap_high = s.quantile([0.01, 0.99])

df_cost["monthly_nomad_cost_fixed"] = s.clip(lower=cap_low, upper=cap_high)

df_cost.sort_values("monthly_nomad_cost_fixed", ascending=False)[
    ["city_name","country_name","monthly_nomad_cost","monthly_nomad_cost_fixed","data_quality_flag"]
].head(10)

,city_name,country_name,monthly_nomad_cost,monthly_nomad_cost_fixed,data_quality_flag
21,New York,United States,4122.760,3180.99845,1
1971,Sunnyvale,United States,3622.385,3180.99845,1
2997,West Hollywood,United States,3261.970,3180.99845,0
2881,North Bethesda,United States,3248.705,3180.99845,0
4224,Monterey,United States,3260.435,3180.99845,0
4190,Saratoga,United States,3632.655,3180.99845,0
3058,Mount Hagen,Papua New Guinea,4168.200,3180.99845,0
3026,Foster City,United States,3452.030,3180.99845,0
1243,Antioch,United States,3552.475,3180.99845,0
2848,Hoboken,United States,3615.645,3180.99845,0


#### **6. Calculamos la variable agregada 5: `local_purchasing_power`**

Poder adquisitivo local relativo al coste de vida nómada.

**Básicamente la variable calcula el promedio del salario neto dividio el coste mensual total: `avg_net_salary / monthly_nomad_cost`**

Lo interesante aquí es saber en cuántas ciudades el salario está por encima de coste o no.

In [114]:
# Cálculo de la relación entre salario neto y coste de vida mensual:

df_cost['local_purchasing_power'] = (
    df_cost['avg_net_salary'] / df_cost['monthly_nomad_cost']
)

# Datos estadísticos generales de la nueva variable agregada: capacidad de compra

print(f"Valores estadísticos de la variable agregada relacionada con capacidad de compra:")
print(f"------------------------------------------------------------------------------------")
print(df_cost['local_purchasing_power'].describe())

# Y calculamos números de ciudades donde el salario está por debajo del coste
# Hay casi 1300 ciudades donde el salario neto promedio no permite subsistir 

print(f"------------------------------------------------------------------------------------")
print(f"Ciudades donde salario > coste: {(df_cost['local_purchasing_power'] > 1).sum()}")
print(f"Ciudades donde salario < coste: {(df_cost['local_purchasing_power'] < 1).sum()}")

Valores estadísticos de la variable agregada relacionada con capacidad de compra:
------------------------------------------------------------------------------------
count    4742.000000
mean        1.491582
std         0.734585
min         0.001476
25%         0.941611
50%         1.443330
75%         1.912883
max         7.864915
Name: local_purchasing_power, dtype: float64
------------------------------------------------------------------------------------
Ciudades donde salario > coste: 3422
Ciudades donde salario < coste: 1320


 #### **7. Calculamos la variable agregada 6: `cappuccino_index`**

El precio del cappuccino varía mucho entre ciudades y puede tener rangos muy distintos. Para poder comparar de forma justa, normalizamos esta
 variable transformando el precio original a un índice en una escala común. Así, el cappuccino_index representa el costo relativo del cappuccino
  en cada ciudad (más alto = más caro, más bajo = más barato), facilitando comparaciones y visualizaciones sin que el análisis se vea dominado
  por la magnitud del precio.
  

In [115]:
#Normalización Min-Max (escala 0 a 1)

#Interpretación: 0 = ciudad más barata, 1 = ciudad más cara.

min_val = df_cost["cappuccino_restaurant"].min()
max_val = df_cost["cappuccino_restaurant"].max()

df_cost["cappuccino_index"] = (df_cost["cappuccino_restaurant"] - min_val) / (max_val - min_val)
df_cost["cappuccino_index"].describe()

count    4742.000000
mean        0.258099
std         0.138229
min         0.000000
25%         0.139059
50%         0.245399
75%         0.360941
max         1.000000
Name: cappuccino_index, dtype: float64

In [116]:
#Aqui volvemos a hacer un top 10 de mas asequibles

df_cost[["city_name","country_name","cappuccino_restaurant","cappuccino_index"]] \
  .sort_values("cappuccino_index", ascending=True).head(10)

,city_name,country_name,cappuccino_restaurant,cappuccino_index
2129,Laghouat,Algeria,0.22,0.000000
2081,Medea,Algeria,0.22,0.000000
1845,Bordj Bou Arreridj,Algeria,0.24,0.002045
2014,Mostaganem,Algeria,0.25,0.003067
2867,Puttalam,Sri Lanka,0.27,0.005112
1614,Moratuwa,Sri Lanka,0.27,0.005112
2858,Ratnapura,Sri Lanka,0.27,0.005112
2137,Batticaloa,Sri Lanka,0.27,0.005112
2568,El Bayadh,Algeria,0.29,0.007157
1381,Djelfa,Algeria,0.30,0.008180


In [117]:
# Top 10 mas caras 

df_cost[["city_name","country_name","cappuccino_restaurant","cappuccino_index"]] \
  .sort_values("cappuccino_index", ascending=False).head(10)

,city_name,country_name,cappuccino_restaurant,cappuccino_index
4355,Gallup,United States,10.00,1.000000
4375,Perrysburg,United States,8.00,0.795501
3291,Plainedge,United States,8.00,0.795501
4677,Menominee,United States,8.00,0.795501
2016,Topeka,United States,7.67,0.761759
1019,Turkmenabat,Turkmenistan,7.14,0.707566
2612,Cheyenne,United States,7.03,0.696319
4541,Big Rapids,United States,7.00,0.693252
4016,Spanish Fork,United States,7.00,0.693252
2405,League City,United States,7.00,0.693252


In [118]:
# Y seguimos aplicando la misma logica

low_price_cappuccino = df_cost["cappuccino_restaurant"].quantile(0.40)
high_price_capuccino = df_cost["cappuccino_restaurant"].quantile(0.60)

df_cost.loc[
    df_cost["cappuccino_restaurant"].between(low_price_cappuccino, high_price_capuccino),
    ["city_name","country_name","cappuccino_restaurant"]
].head(20)

,city_name,country_name,cappuccino_restaurant
3,Mumbai,India,2.48
7,Jakarta,Indonesia,2.23
10,Bangkok,Thailand,2.13
14,Mexico City,Mexico,2.71
20,Manila,Philippines,2.97
29,Wuhan,China,2.86
33,Nagoya,Japan,2.65
36,Baoding,China,2.85
37,Lima,Peru,2.66
40,Nanyang,China,2.73


#### **8. Calculamos la variable agregada 7: `housing_salary_ratio`**

Esta variable resume la relación entre el coste de la vivienda (alquiler) y el salario, un factor relevante para evaluar el atractivo económico
 de un destino para nómadas digitales. En el análisis exploratorio, el alquiler suele ser uno de los componentes con mayor impacto dentro del
 coste de vida, por encima de categorías como alimentación o transporte. Para ello utilizaremos la variable `nomad_housing_cost`
  Los datos salary se obtienen del dataset de Kaggle basado en valores publicados por Numbeo. Por ello, `avg_net_salary` se interpreta como una aproximación del salario mensual neto promedio (after tax) a nivel de ciudad. [Salary-Cost of Living](https://www.numbeo.com/cost-of-living/)

In [ ]:
# Identificamos columnas que contengan informacion de salarios

[c for c in df_cost.columns if "salary" in c.lower() or "wage" in c.lower() or "income" in c.lower()]

['avg_net_salary']

In [119]:
#Filtramos un encabezado de sueldos por País, Ciudad.

df_cost[["city_name", "country_name", "avg_net_salary"]].head(20)

,city_name,country_name,avg_net_salary
0,Seoul,South Korea,2689.62
1,Shanghai,China,1419.87
2,Guangzhou,China,1211.68
3,Mumbai,India,640.81
4,Delhi,India,586.46
5,Dhaka,Bangladesh,280.73
6,Osaka,Japan,2322.46
7,Jakarta,Indonesia,509.12
8,Shenzhen,China,1572.22
9,Kinshasa,Congo,400.00


In [120]:
# Sumamos las variables de alquiler mas los gastos por mes teniendo en cuenta que el alquiler es un gasto mensual

df_cost["housing_plus_food_cost"] = df_cost["nomad_housing_cost"] + monthly_food_cost
df_cost["housing_plus_food_cost"].describe()


count     4742.000000
mean       915.642920
std        666.620773
min         89.975000
25%        387.206250
50%        735.087500
75%       1338.363750
max      10955.410000
Name: housing_plus_food_cost, dtype: float64

In [121]:

df_cost["housing_salary_ratio"] = np.where(
    df_cost["avg_net_salary"] > 0,
    df_cost["housing_plus_food_cost"] / df_cost["avg_net_salary"] * 100,
    np.nan
)

df_cost["housing_salary_ratio"].describe()

count    4742.000000
mean       75.380152
std       112.653632
min         7.712425
25%        42.677946
50%        53.852494
75%        80.950096
max      3136.083333
Name: housing_salary_ratio, dtype: float64

In [123]:
df_cost[["city_name","country_name","housing_salary_ratio", "avg_net_salary"]].head(20)

,city_name,country_name,housing_salary_ratio,avg_net_salary
0,Seoul,South Korea,39.711558,2689.62
1,Shanghai,China,76.271771,1419.87
2,Guangzhou,China,50.877707,1211.68
3,Mumbai,India,82.881821,640.81
4,Delhi,India,52.546636,586.46
5,Dhaka,Bangladesh,80.189150,280.73
6,Osaka,Japan,35.819993,2322.46
7,Jakarta,Indonesia,106.489629,509.12
8,Shenzhen,China,52.011487,1572.22
9,Kinshasa,Congo,447.487500,400.00


In [125]:
# Convertimos a porcentaje
df_cost["housing_salary_pct"] = df_cost["housing_salary_ratio"] * 100
df_cost[["city_name","country_name","housing_salary_pct","avg_net_salary"]].head(10)

,city_name,country_name,housing_salary_pct,avg_net_salary
0,Seoul,South Korea,3971.155777,2689.62
1,Shanghai,China,7627.177136,1419.87
2,Guangzhou,China,5087.770699,1211.68
3,Mumbai,India,8288.182144,640.81
4,Delhi,India,5254.663575,586.46
5,Dhaka,Bangladesh,8018.914972,280.73
6,Osaka,Japan,3581.999259,2322.46
7,Jakarta,Indonesia,10648.962916,509.12
8,Shenzhen,China,5201.148694,1572.22
9,Kinshasa,Congo,44748.750000,400.00


In [126]:
df_cost[["city_name","country_name","monthly_nomad_cost"]].head(21)

,city_name,country_name,monthly_nomad_cost
0,Seoul,South Korea,1314.950
1,Shanghai,China,1194.500
2,Guangzhou,China,721.255
3,Mumbai,India,588.925
4,Delhi,India,385.855
5,Dhaka,Bangladesh,302.605
6,Osaka,Japan,1081.635
7,Jakarta,Indonesia,668.950
8,Shenzhen,China,939.165
9,Kinshasa,Congo,2673.940


In [ ]:
# 1) Comida mensual estimada (8 comidas fuera/mes + canasta súper)
df_cost["monthly_food_cost_est"] = df_cost["daily_meal_cost"] * 8 + df_cost["basic_basket_monthly_est"]

# 2) Vivienda + comida
df_cost["housing_plus_food_cost"] = df_cost["nomad_housing_cost"] + df_cost["monthly_food_cost_est"]

# 3) % del salario destinado a vivienda+comida
df_cost["housing_salary_ratio"] = np.where(
    df_cost["avg_net_salary"] > 0,
    df_cost["housing_plus_food_cost"] / df_cost["avg_net_salary"] * 100,
    np.nan
)

# 4) tratar outliers para análisis (cap p99)
cap = df_cost["housing_salary_ratio"].quantile(0.99)
df_cost["housing_salary_ratio_fixed"] = df_cost["housing_salary_ratio"].clip(upper=cap)

df_cost["housing_salary_ratio_fixed"].describe()

# Esto es lo màs limpio que pude hacer como conclusion

count    4742.000000
mean       70.609358
std        54.962644
min         7.712425
25%        42.677946
50%        53.852494
75%        80.950096
max       406.474188
Name: housing_salary_ratio_fixed, dtype: float64

### *La interpretacion de los datos obtenidos en esta variable no son muy confiables y habria que buscar un dataset que nos muestre rangos mas confiables ya que los datos que extrajo el autor del dataset Cost Of Living para indicar sueldos, proviene de una pagina la cual los datos se basan en una opinion publica y no en un dato estadistico real* ###